In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install langchain-community langchain --quiet
!pip install unstructured chromadb tiktoken --quiet
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers --q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.9.0 requires keras-core, which is not installed.
keras-nlp 0.12.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.2 which is incompatible.
cud

Importing the necessary libraries

In [2]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [3]:
from langchain_community.document_loaders import DirectoryLoader

Loading the data to Langchain loader. The DirectoryLoader is used, since the complete text files are present in a directory.

In [5]:
loader = DirectoryLoader("/kaggle/input/rags-txt-docs/Rags_txt_file", glob="**/*.txt")
docs = loader.load()
len(docs)

50

In [6]:
print(docs[0].page_content[:1000])

Mistral 7B

Albert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford,

Devendra Singh Chaplot, Diego de las Casas, Florian Bressand, Gianna Lengyel,

Guillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, Marie-Anne Lachaux,

Pierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix,

William El Sayed

Abstract

We introduce Mistral 7B, a 7–billion-parameter language model engineered for

superior performance and efficiency. Mistral 7B outperforms the best open 13B

model (Llama 2) across all evaluated benchmarks, and the best released 34B

model (Llama 1) in reasoning, mathematics, and code generation. Our model

leverages grouped-query attention (GQA) for faster inference, coupled with sliding

window attention (SWA) to effectively handle sequences of arbitrary length with a

reduced inference cost. We also provide a model fine-tuned to follow instructions,

Mistral 7B – Instruct, that surpasses Llama 2 13B – chat model both on human and

automated benc

In [4]:
!pip install langchain-chroma --q

The text splitter is used to split the loaded documents. This is because we want to create the embedding vectors of these documents, but most of the embedding models can handle embeddings of smaller dimensions. Thus, we create chunks of these documents, and allow small overlap of these chunks so that the semantic relation is preserved to some extent.

In [8]:
splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=30)

chunked_docs = splitter.split_documents(docs)

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="avsolatorio/GIST-Embedding-v0")

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
2024-06-29 11:47:44.148589: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 11:47:44.148716: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 11:47:44.273305: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS fact

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/117k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

The splitted chunks are then embedded using hugging face embeddings and then stored in chroma vetor store.

In [7]:
# import
from langchain_chroma import Chroma

db = Chroma.from_documents(chunked_docs, embeddings, collection_name="chroma_embeddings", persist_directory="./chroma_embedding_db")

Based on **similarity score**, top 7 embedding vectors are retreived using the retreiver from the vector store.

In [13]:
vector_retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 7})

In [14]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [27]:
# docs = vector_retriever.get_relevant_documents(query="What are the layers in multi head attention?")
# pretty_print_docs(docs)

In [15]:
from langchain.llms import HuggingFaceHub
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

In [17]:
import os
from getpass import getpass

os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("Enter HuggingFace Hub Token:")

Enter HuggingFace Hub Token: ·····································


In [18]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

 ········································


In [20]:
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage

In [21]:
chat = ChatCohere(model="command")

The **LLMChainExtractor** extracts key information from documents using a language model, identifying sections most relevant to the user's query. The **ContextualCompressionRetriever** combines a base retriever, which searches for relevant documents, and refines these documents by extracting the most pertinent parts. The **EmbeddingsFilter** enhances this process by using embeddings to filter documents, selecting those that are semantically similar to the query. Together, these steps improve the RAG system by focusing on the most relevant information, enhancing both the accuracy and efficiency of responses, and ensuring the language model works with high-quality, contextually relevant data.

In [22]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

#creating the compressor
compressor = LLMChainExtractor.from_llm(llm=chat)

#compressor retriver = base retriever + compressor
compression_retriever = ContextualCompressionRetriever(base_retriever=vector_retriever,
                                                       base_compressor=compressor)

In [23]:
from getpass import getpass
import os
# from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.document_compressors import EmbeddingsFilter

# os.environ["OPENAI_API_KEY "] = getpass()
embdeddings_filter = EmbeddingsFilter(embeddings=embeddings)
compression_retriever_filter = ContextualCompressionRetriever(base_retriever=vector_retriever,
                                                       base_compressor=embdeddings_filter)

# compressed_docs = compression_retriever_filter.get_relevant_documents(query="How does Mistral outperform Llama?")
# pretty_print_docs(compressed_docs)

In [24]:
print(type(compression_retriever_filter))

<class 'langchain.retrievers.contextual_compression.ContextualCompressionRetriever'>


In [25]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [35]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
# from langchain_experimental.chat_models import Llama2Chat

In [47]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

template_messages = [
    SystemMessage(content="You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{text}"),
]
prompt_template = ChatPromptTemplate.from_messages(template_messages)

In [50]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(llm=chat, prompt=prompt_template, memory=memory)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


The **Contextualize Prompt** reformulates user questions to be clear and standalone, ensuring queries are understood with prior chat history. The **System Prompt** guides the assistant to answer queries by instructing the llm to refer the information from the given context, and do not generate answers on its own.

The **ChatPromptTemplate** combines system instructions, chat history, and the latest user input into a structured prompt, helping the model understand and generate accurate responses.

In [51]:
CONTEXTUALIZE_PROMPT = """Given a chat history and the latest user question which \
might reference context in the chat history, formulate a standalone question which can be \
understood without the chat history. Do NOT answer the question, just reformulate it if needed \
and otherwise return it as is.

Chat History:

{chat_history}"""

SYSTEM_PROMPT2 = """
You have to answer in the same language as the question. \
First determine in which language is the question.

You are a specialist in Artificial Intelligence. You have to assist the users with their\
question. You first have to search answers in the "Knowledge Base". If no answers are found \
in the "Knowledge Base", then reply "Sorry, it seems I missed this in my training...".

Knowledge Base:

{context}

Chat History:

{chat_history}
"""

In [38]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

**History-Aware Retriever** integrates previous chat history with the current query to improve the relevance of the retrieved documents, ensuring contextual awareness. This component improve the RAG system by ensuring the assistant is contextually aware and capable of providing precise and relevant answers, leading to a more coherent and helpful conversational experience.

In [39]:
history_aware_retriever = create_history_aware_retriever(
    chat, vector_retriever, contextualize_q_prompt
)

The **Question Answer Chain** uses the retrieved context and chat history to generate concise answers, ensuring responses are based on the most relevant information. 

In [43]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(chat, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

`ChatMessageHistory` and `BaseChatMessageHistory` manage the chat histories. The `get_session_history` function retrieves or creates a chat history for each session ID. The `RunnableWithMessageHistory` wraps the RAG chain, incorporating session-specific chat history into the process. 

In [44]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [45]:
conversational_rag_chain.invoke(
    {"input": "Tell me about GPT 4"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"GPT-4 is a language model created by OpenAI, which is a non-profit research company that develops and promotes the use of AI technologies to benefit humanity. It is the latest in a series of Generative Pre-trained Transformer (GPT) models, which are known for their ability to understand and generate human-like language. \n\nGPT-4 has several improvements and updates compared to its predecessor, GPT-3.5. Firstly, it is more reliable, creative, and able to handle much more nuanced instructions, which makes it particularly useful in language tasks that require adaptability and creativity. It has better performance in generating text that is more coherent, less repetitive, and contains less factual errors. Additionally, it excels in tasks involving common sense reasoning, command simulation, and question answering. \n\nOne of the most significant advancements in GPT-4 is its performance in multilingual tasks, where it outperforms GPT-3.5 in 24 out of 26 languages tested. This makes it a s

In [46]:
conversational_rag_chain.invoke(
    {"input": "Tell me how it outperfroms other models"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"GPT-4 excels in a wide range of natural language processing tasks compared to other language models due to several key factors. Firstly, its capacity enables the model to capture long-range dependencies in sentences, allowing it to generate more coherent and contextually appropriate responses. This helps improve its understanding and generation of language in a variety of settings. \n\nSecondly, GPT-4 boasts impressive performance in low-resource languages. Unlike other models that struggle with low-resource languages due to the lack of extensive training data, GPT-4 excels even in these scenarios. This is because the model is pre-trained on a vast amount of data in multiple languages, enabling it to develop a general understanding of language patterns and semantics. \n\nAdditionally, GPT-4 performs better at generating text that is more coherent, less repetitive, and contains less factual errors. This is largely due to the model's Generative Adversarial Network (GAN) architecture, wh